# Phase 3: CICY - Quintic Hypersurface

**Objectif**: Tester λ₁ × H* = 6 sur le célèbre Quintic

**Runtime**: GPU A100 recommandé (N > 20000)

---

## Le Quintic de Fermat

Le Quintic est défini comme hypersurface dans $\mathbb{CP}^4$:

$$X = \{[z_0 : z_1 : z_2 : z_3 : z_4] \in \mathbb{CP}^4 \;|\; p(z) = 0\}$$

où $p(z) = z_0^5 + z_1^5 + z_2^5 + z_3^5 + z_4^5$ (Fermat)

ou plus généralement:
$p(z) = z_0^5 + z_1^5 + z_2^5 + z_3^5 + z_4^5 - 5\psi z_0 z_1 z_2 z_3 z_4$

### Propriétés

| Propriété | Valeur |
|-----------|--------|
| Holonomie | **SU(3)** |
| h¹¹ | 1 |
| h²¹ | 101 |
| χ | -200 |
| dim réelle | 6 |

### Défi: Métrique Inconnue

Le Quintic n'a **pas de métrique Ricci-flat explicite**. Options:
1. **Métrique ambiante (Fubini-Study)** - approximation grossière
2. **Algorithme de Donaldson** - itératif, coûteux
3. **Réseaux de neurones (PINN)** - récent, prometteur

Pour cette première exploration, on utilise la métrique ambiante Fubini-Study.

---

In [ ]:
# Cell 1: Setup
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import eigsh
from scipy.optimize import minimize
import json
from datetime import datetime
import time
import matplotlib.pyplot as plt

# GPU optionnel
try:
    import cupy as cp
    GPU_AVAILABLE = True
    print("✓ CuPy available - GPU acceleration enabled")
except ImportError:
    GPU_AVAILABLE = False
    print("✗ CuPy not available - using CPU")

print(f"NumPy version: {np.__version__}")

In [ ]:
# Cell 2: Quintic Constants

# Hodge numbers
H11, H21 = 1, 101
CHI = -200  # χ = 2(h¹¹ - h²¹)

# H* definitions
HSTAR_A = H11 + H21 + 2       # = 104
HSTAR_B = H11 + 2*H21 + 4     # = 207
HSTAR_C = abs(H11 - H21) + 2  # = 102

# Conjecture target
TARGET = 6

# Target λ₁ for each H* definition
LAMBDA1_TARGET_A = TARGET / HSTAR_A  # ≈ 0.0577
LAMBDA1_TARGET_B = TARGET / HSTAR_B  # ≈ 0.0290
LAMBDA1_TARGET_C = TARGET / HSTAR_C  # ≈ 0.0588

# Numerical parameters
N_VALUES = [2000, 5000, 10000, 20000]
K_VALUES = [25, 40, 60, 100]
SEEDS = [42, 123, 456]

# Fermat quintic parameter (psi=0 for Fermat, psi≠1 for deformations)
PSI = 0.0

# PASS/FAIL criteria
TOLERANCE = 0.20  # 20% for numerical CY (harder than orbifold)

print(f"Quintic Configuration:")
print(f"  Hodge numbers: h¹¹={H11}, h²¹={H21}")
print(f"  Euler: χ={CHI}")
print(f"  Holonomy: SU(3)")
print(f"\nH* definitions:")
print(f"  A: {HSTAR_A} → λ₁ target = {LAMBDA1_TARGET_A:.5f}")
print(f"  B: {HSTAR_B} → λ₁ target = {LAMBDA1_TARGET_B:.5f}")
print(f"  C: {HSTAR_C} → λ₁ target = {LAMBDA1_TARGET_C:.5f}")
print(f"\nNote: H* ~ 100 means λ₁ ~ 0.06, very small!")

In [ ]:
# Cell 3: Quintic Sampling Functions

def quintic_polynomial(z: np.ndarray, psi: float = 0.0) -> np.ndarray:
    """
    Evaluate the quintic polynomial:
    p(z) = z₀⁵ + z₁⁵ + z₂⁵ + z₃⁵ + z₄⁵ - 5ψ z₀z₁z₂z₃z₄
    
    Args:
        z: (N, 5) complex array of homogeneous coordinates
        psi: deformation parameter (0 for Fermat)
    """
    z5_sum = np.sum(z**5, axis=1)
    prod_term = 5 * psi * np.prod(z, axis=1)
    return z5_sum - prod_term


def quintic_gradient(z: np.ndarray, psi: float = 0.0) -> np.ndarray:
    """
    Gradient of quintic polynomial: ∇p = (5z₀⁴, 5z₁⁴, 5z₂⁴, 5z₃⁴, 5z₄⁴) - 5ψ ∏(z_j≠i)
    """
    N = z.shape[0]
    grad = 5 * z**4  # (N, 5)
    
    if psi != 0:
        prod_all = np.prod(z, axis=1, keepdims=True)  # (N, 1)
        # ∂/∂z_i of ∏ z_j = ∏_{j≠i} z_j = prod_all / z_i
        with np.errstate(divide='ignore', invalid='ignore'):
            grad -= 5 * psi * prod_all / z
            grad = np.nan_to_num(grad, nan=0.0, posinf=0.0, neginf=0.0)
    
    return grad


def project_to_quintic(z: np.ndarray, psi: float = 0.0, 
                       max_iter: int = 20, tol: float = 1e-10) -> np.ndarray:
    """
    Project point onto quintic hypersurface using Newton iteration.
    """
    for _ in range(max_iter):
        p = quintic_polynomial(z, psi)
        if np.all(np.abs(p) < tol):
            break
        
        grad = quintic_gradient(z, psi)
        grad_norm_sq = np.sum(np.abs(grad)**2, axis=1, keepdims=True)
        grad_norm_sq = np.maximum(grad_norm_sq, 1e-20)
        
        # Newton step: z_new = z - p * grad* / |grad|²
        z = z - (p[:, None] * np.conj(grad)) / grad_norm_sq
        
        # Normalize in CP⁴
        z = z / np.linalg.norm(z, axis=1, keepdims=True)
    
    return z


def sample_quintic(N: int, psi: float = 0.0, seed: int = 42, 
                   max_attempts: int = None) -> np.ndarray:
    """
    Sample N points on the quintic hypersurface.
    
    Method: rejection sampling in CP⁴, project to hypersurface.
    """
    rng = np.random.default_rng(seed)
    if max_attempts is None:
        max_attempts = N * 10
    
    points = []
    attempts = 0
    batch_size = min(N * 2, 10000)
    
    while len(points) < N and attempts < max_attempts:
        # Random points in CP⁴ (complex Gaussian, then normalize)
        z = rng.standard_normal((batch_size, 5)) + 1j * rng.standard_normal((batch_size, 5))
        z = z / np.linalg.norm(z, axis=1, keepdims=True)
        
        # Project onto quintic
        z_proj = project_to_quintic(z, psi)
        
        # Verify on hypersurface
        residual = np.abs(quintic_polynomial(z_proj, psi))
        valid = residual < 1e-8
        
        points.extend(z_proj[valid])
        attempts += batch_size
    
    points = np.array(points[:N])
    
    if len(points) < N:
        print(f"Warning: only got {len(points)}/{N} valid points")
    
    return points


print("✓ Quintic sampling functions defined")

In [ ]:
# Cell 4: Fubini-Study Distance

def fubini_study_distance(z1: np.ndarray, z2: np.ndarray) -> np.ndarray:
    """
    Fubini-Study distance on CP⁴.
    
    d_FS(z1, z2) = arccos(|<z1, z2>| / (|z1| |z2|))
    
    For normalized z: d_FS = arccos(|<z1, z2>|)
    
    Returns: (N1, N2) distance matrix
    """
    # Hermitian inner product: <z1, z2> = sum(z1_i * conj(z2_i))
    # For (N1, 5) @ (5, N2) we need: z1 @ z2.conj().T
    inner = z1 @ z2.conj().T
    
    # |<z1, z2>| clipped to [0, 1]
    cos_dist = np.clip(np.abs(inner), 0, 1)
    
    # Fubini-Study distance
    return np.arccos(cos_dist).astype(np.float32)


def fubini_study_distance_batched(z: np.ndarray, batch_size: int = 5000) -> np.ndarray:
    """
    Batched FS distance for large N.
    """
    N = z.shape[0]
    D = np.zeros((N, N), dtype=np.float32)
    
    for i in range(0, N, batch_size):
        end_i = min(i + batch_size, N)
        for j in range(0, N, batch_size):
            end_j = min(j + batch_size, N)
            D[i:end_i, j:end_j] = fubini_study_distance(z[i:end_i], z[j:end_j])
    
    return D


print("✓ Fubini-Study distance functions defined")

In [ ]:
# Cell 5: Graph Laplacian

def build_graph_laplacian(D: np.ndarray, k: int = 25,
                          laplacian_type: str = "symmetric") -> sp.csr_matrix:
    """
    Build graph Laplacian from distance matrix.
    """
    N = D.shape[0]
    k = min(k, N - 1)
    
    # Sigma = median of k-NN distances
    knn_dists = np.partition(D, k, axis=1)[:, 1:k+1]
    sigma = max(np.median(knn_dists), 1e-10)
    
    # Gaussian weights
    W = np.exp(-D**2 / (2 * sigma**2))
    np.fill_diagonal(W, 0)
    
    # Keep only k nearest neighbors
    for i in range(N):
        idx_keep = np.argpartition(-W[i], k)[:k]
        mask = np.ones(N, dtype=bool)
        mask[idx_keep] = False
        W[i, mask] = 0
    
    # Symmetrize
    W = (W + W.T) / 2
    
    # Degrees
    d = np.maximum(W.sum(axis=1), 1e-10)
    
    # Build Laplacian
    if laplacian_type == "symmetric":
        d_inv_sqrt = 1.0 / np.sqrt(d)
        L = np.eye(N) - np.outer(d_inv_sqrt, d_inv_sqrt) * W
    else:
        L = np.diag(d) - W
    
    return sp.csr_matrix(L)


def compute_lambda1(L: sp.csr_matrix, n_eigenvalues: int = 5) -> float:
    """
    Compute λ₁ (first non-zero eigenvalue).
    """
    try:
        eigs, _ = eigsh(L, k=n_eigenvalues, which='SM', tol=1e-8)
        eigs = np.sort(np.real(eigs))
        
        for ev in eigs:
            if ev > 1e-8:
                return float(ev)
        
        return float(eigs[1]) if len(eigs) > 1 else 0.0
        
    except Exception as e:
        print(f"Eigensolve error: {e}")
        return np.nan


print("✓ Laplacian functions ready")

In [ ]:
# Cell 6: Quick Test
print("=" * 60)
print("  QUICK TEST: N=2000")
print("=" * 60)

t0 = time.time()

# Sample on Quintic
z = sample_quintic(2000, psi=PSI, seed=42)
print(f"Sampled {len(z)} points on Quintic")

# Verify on hypersurface
residuals = np.abs(quintic_polynomial(z, PSI))
print(f"Residuals: max={residuals.max():.2e}, mean={residuals.mean():.2e}")

# Distance matrix
D = fubini_study_distance_batched(z)
print(f"Distance matrix: {D.shape}, range [{D.min():.4f}, {D.max():.4f}]")

# Laplacian & eigenvalue
L = build_graph_laplacian(D, k=40)
lambda1 = compute_lambda1(L)

print(f"\nResults:")
print(f"  λ₁ (measured): {lambda1:.5f}")
print(f"  Time: {time.time() - t0:.2f}s")

print(f"\nConjecture test (λ₁ × H* = 6):")
for name, Hstar, target in [("A", HSTAR_A, LAMBDA1_TARGET_A), 
                             ("B", HSTAR_B, LAMBDA1_TARGET_B),
                             ("C", HSTAR_C, LAMBDA1_TARGET_C)]:
    product = lambda1 * Hstar
    deviation = abs(product - TARGET) / TARGET * 100
    status = "✓" if deviation < TOLERANCE * 100 else "?"
    print(f"  H*_{name}={Hstar}: λ₁×H* = {product:.3f} (dev={deviation:.1f}%) [{status}]")

In [ ]:
# Cell 7: Full Grid Run
print("=" * 60)
print("  FULL VALIDATION: QUINTIC")
print("=" * 60)
print(f"Grid: N={N_VALUES}, k={K_VALUES}")
print(f"Seeds: {SEEDS}")
print()

results = []
t_start = time.time()

for N in N_VALUES:
    print(f"\nN = {N}...")
    
    for seed in SEEDS:
        # Sample once per (N, seed)
        t0 = time.time()
        z = sample_quintic(N, psi=PSI, seed=seed)
        D = fubini_study_distance_batched(z)
        t_sample = time.time() - t0
        
        for k in K_VALUES:
            t0 = time.time()
            L = build_graph_laplacian(D, k=k)
            lambda1 = compute_lambda1(L)
            t_eig = time.time() - t0
            
            # Test all H* definitions
            for Hstar_name, Hstar in [("A", HSTAR_A), ("B", HSTAR_B), ("C", HSTAR_C)]:
                product = lambda1 * Hstar
                deviation = abs(product - TARGET) / TARGET * 100
                
                result = {
                    "N": N,
                    "k": k,
                    "seed": seed,
                    "Hstar_def": Hstar_name,
                    "Hstar": Hstar,
                    "lambda1": float(lambda1),
                    "product": float(product),
                    "deviation_from_6_pct": float(deviation),
                    "passed": deviation < TOLERANCE * 100,
                    "time_sample_s": t_sample,
                    "time_eig_s": t_eig,
                }
                results.append(result)
        
        # Print summary line
        products = [lambda1 * H for H in [HSTAR_A, HSTAR_B, HSTAR_C]]
        print(f"  seed={seed}: λ₁={lambda1:.5f} → products(A/B/C)={[f'{p:.2f}' for p in products]}")

t_total = time.time() - t_start
print(f"\nTotal time: {t_total/60:.1f} minutes")

In [ ]:
# Cell 8: Analysis
print("=" * 60)
print("  QUINTIC ANALYSIS")
print("=" * 60)

# Group by H* definition
analysis = {}
for Hstar_name in ["A", "B", "C"]:
    subset = [r for r in results if r["Hstar_def"] == Hstar_name]
    products = [r["product"] for r in subset]
    
    # High-N only
    high_N_subset = [r for r in subset if r["N"] >= 10000]
    high_N_products = [r["product"] for r in high_N_subset] if high_N_subset else products
    
    analysis[Hstar_name] = {
        "Hstar": subset[0]["Hstar"],
        "mean_product": np.mean(products),
        "std_product": np.std(products),
        "high_N_mean_product": np.mean(high_N_products),
        "n_passed": sum(1 for r in subset if r["passed"]),
        "n_total": len(subset),
    }

print(f"\n| H* Def | H* | Mean Product | Std | High-N Mean | Pass Rate |")
print(f"|--------|-----|--------------|-----|-------------|-----------|")
for name, data in analysis.items():
    pass_rate = data["n_passed"] / data["n_total"] * 100
    print(f"| {name}      | {data['Hstar']:3d} | {data['mean_product']:12.3f} | {data['std_product']:.3f} | {data['high_N_mean_product']:11.3f} | {pass_rate:8.1f}% |")

# Winner
winner = min(analysis.keys(), key=lambda k: abs(analysis[k]["high_N_mean_product"] - TARGET))
print(f"\n→ Best match: H*_{winner} (closest to {TARGET})")

In [ ]:
# Cell 9: Visualization
print("Generating plots...")

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
colors = {'A': 'blue', 'B': 'green', 'C': 'orange'}

# Plot 1: λ₁ vs k (for different N)
ax1 = axes[0]
for N in [5000, 10000, 20000]:
    subset = [r for r in results if r["N"] == N and r["Hstar_def"] == "A"]
    if subset:
        by_k = {}
        for r in subset:
            by_k.setdefault(r["k"], []).append(r["lambda1"])
        ks = sorted(by_k.keys())
        means = [np.mean(by_k[k]) for k in ks]
        ax1.plot(ks, means, 'o-', label=f'N={N}')

ax1.axhline(y=LAMBDA1_TARGET_A, color='r', linestyle='--', label=f'Target (A): {LAMBDA1_TARGET_A:.4f}')
ax1.set_xlabel('k (neighbors)')
ax1.set_ylabel('λ₁')
ax1.set_title('Quintic: λ₁ vs k')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Products by H* definition
ax2 = axes[1]
for Hstar_name in ["A", "B", "C"]:
    subset = [r for r in results if r["Hstar_def"] == Hstar_name]
    by_N = {}
    for r in subset:
        by_N.setdefault(r["N"], []).append(r["product"])
    Ns = sorted(by_N.keys())
    means = [np.mean(by_N[N]) for N in Ns]
    ax2.plot(Ns, means, 'o-', color=colors[Hstar_name], label=f'H*_{Hstar_name}')

ax2.axhline(y=TARGET, color='r', linestyle='--', linewidth=2, label=f'Target = {TARGET}')
ax2.axhspan(TARGET * (1 - TOLERANCE), TARGET * (1 + TOLERANCE), alpha=0.2, color='green')
ax2.set_xlabel('N')
ax2.set_ylabel('λ₁ × H*')
ax2.set_title('Quintic: Conjecture Test')
ax2.set_xscale('log')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Comparison with target lines
ax3 = axes[2]
# Box plot by definition
data_for_box = []
labels = []
for Hstar_name in ["A", "B", "C"]:
    subset = [r for r in results if r["Hstar_def"] == Hstar_name and r["N"] >= 5000]
    data_for_box.append([r["product"] for r in subset])
    labels.append(f'H*_{Hstar_name}={analysis[Hstar_name]["Hstar"]}')

bp = ax3.boxplot(data_for_box, labels=labels, patch_artist=True)
for patch, color in zip(bp['boxes'], [colors['A'], colors['B'], colors['C']]):
    patch.set_facecolor(color)
    patch.set_alpha(0.5)

ax3.axhline(y=TARGET, color='r', linestyle='--', linewidth=2, label='Target = 6')
ax3.set_ylabel('λ₁ × H*')
ax3.set_title('Quintic: Distribution by H* Definition')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('Quintic_validation.png', dpi=150)
plt.show()

print("✓ Saved: Quintic_validation.png")

In [ ]:
# Cell 10: Summary
print("=" * 60)
print("  PHASE 3 SUMMARY: QUINTIC")
print("=" * 60)

best_def = winner
best_data = analysis[best_def]
best_deviation = abs(best_data['high_N_mean_product'] - TARGET) / TARGET * 100

print(f"\nManifold: Quintic P⁴[5]")
print(f"  Holonomy: SU(3)")
print(f"  h¹¹={H11}, h²¹={H21}, χ={CHI}")
print(f"  Métrique: Fubini-Study (ambient, approximation)")

print(f"\nConjecture: λ₁ × H* = {TARGET}")

print(f"\nBest H* definition: {best_def}")
print(f"  H* = {best_data['Hstar']}")
print(f"  High-N mean product: {best_data['high_N_mean_product']:.3f}")
print(f"  Deviation from {TARGET}: {best_deviation:.1f}%")

# PASS/FAIL
phase3_passed = best_deviation < TOLERANCE * 100

print("\n" + "-" * 40)
if phase3_passed:
    print(f"  PHASE 3 STATUS: [PASS]")
    print(f"  Quintic satisfies λ₁ × H*_{best_def} ≈ {TARGET}")
else:
    print(f"  PHASE 3 STATUS: [INCONCLUSIVE]")
    print(f"  Deviation {best_deviation:.1f}% > tolerance {TOLERANCE*100:.0f}%")
    print(f"  Note: Using Fubini-Study metric (not true Ricci-flat)")
    print(f"  Consider: Donaldson algorithm or neural network metric")
print("-" * 40)

In [ ]:
# Cell 11: Save Results
output = {
    "metadata": {
        "phase": "Phase 3: Quintic",
        "timestamp": datetime.now().isoformat(),
        "manifold": "Quintic",
        "holonomy": "SU(3)",
        "h11": H11,
        "h21": H21,
        "chi": CHI,
        "psi": PSI,
        "metric": "Fubini-Study (ambient)",
        "conjecture_target": TARGET,
        "Hstar_definitions": {
            "A": {"formula": "h11 + h21 + 2", "value": HSTAR_A},
            "B": {"formula": "h11 + 2*h21 + 4", "value": HSTAR_B},
            "C": {"formula": "|h11 - h21| + 2", "value": HSTAR_C},
        },
        "N_values": N_VALUES,
        "k_values": K_VALUES,
        "seeds": SEEDS,
        "tolerance": TOLERANCE,
        "total_time_minutes": t_total / 60,
    },
    "results": results,
    "analysis": analysis,
    "summary": {
        "best_Hstar_definition": best_def,
        "best_Hstar_value": best_data["Hstar"],
        "best_mean_product": best_data["high_N_mean_product"],
        "best_deviation_pct": best_deviation,
        "phase3_passed": phase3_passed,
    },
    "caveats": [
        "Using Fubini-Study (ambient) metric, not true Ricci-flat",
        "For better results, consider Donaldson algorithm or neural network metric",
        "H* ~ 100 means very small λ₁ ~ 0.06, harder to measure accurately",
    ],
}

filename = "Quintic_validation_results.json"
with open(filename, "w") as f:
    json.dump(output, f, indent=2)

print(f"\n✓ Saved: {filename}")
print("\nDownload this file and share with Claude!")

In [ ]:
# Cell 12: Final Cross-Comparison
print("\n" + "=" * 60)
print("  UNIVERSAL LAW: G₂ vs SU(3)")
print("=" * 60)

# All results so far
COMPARISON = [
    {
        "manifold": "K₇ (TCS)",
        "holonomy": "G₂",
        "dim_Hol": 14,
        "h": 1,
        "target": 13,
        "measured": 13.0,  # From N=50000 validation
        "status": "EXACT",
    },
    {
        "manifold": "T⁶/ℤ₃",
        "holonomy": "SU(3)",
        "dim_Hol": 8,
        "h": 2,
        "target": 6,
        "measured": "??? (Phase 2)",
        "status": "PENDING",
    },
    {
        "manifold": "Quintic",
        "holonomy": "SU(3)",
        "dim_Hol": 8,
        "h": 2,
        "target": 6,
        "measured": best_data["high_N_mean_product"],
        "status": "PASS" if phase3_passed else "INCONCLUSIVE",
    },
]

print(f"\nConjecture: λ₁ × H* = dim(Hol) - h")
print()
print(f"| Manifold | Hol | dim | h | Target | Measured | Status |")
print(f"|----------|-----|-----|---|--------|----------|--------|")
for r in COMPARISON:
    meas = f"{r['measured']:.2f}" if isinstance(r['measured'], float) else r['measured']
    print(f"| {r['manifold']:<10} | {r['holonomy']} | {r['dim_Hol']:3d} | {r['h']} | {r['target']:6d} | {meas:>8} | {r['status']} |")

print(f"\n=> Run Phase 2 notebook to complete T⁶/ℤ₃ results")
print(f"=> If both CY₃ give λ₁×H* ≈ 6, the universal law is strongly supported!")